# 텍스트 데이터 처리
이번 예제에서는 스탠포드 대학의 [자연어처리랩](http://nlp.stanford.edu/)에서 제공하는 [교재](http://nlp.stanford.edu/IR-book/)를 사용하여 
[텍스트처리](http://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html)를 소개하겠다. 여기서는 단어의 어근을 찾고 단어의 기본형을 처리하는 방법을 소개한다.

---
###### 어근과 기본형

언어에서는 여러가지 표현을 하기 위해서 기본 단어의 다양한 변형된 형태를 사용한다. 예를 들어 organize, organizes, organizing 등이 사용된다. 또한 비슷한 의미를 나타내기 위해서 유사한 단어들이 사용된다. 텍스트를 처리하기 위해서는 이렇게 서로 관련이 있는 단어들을 찾아내거나 변형하는 작업이 자주 필요하다. 

단어의 어근을 찾거나 기본형을 찾는 것은 굴절어의 표현을 종류를 줄이거나 단어의 변형된 형식의 숫자를 줄여서 기본형을 만들어 데이터 분석을 용이하게 하기 위해서 필요하다. 예를 들어 다음과 같은 작업이 필요한 경우가 많다.

am, are, is ---> be  
car, cars, car's, cars' ---> car

이와같은 단어의 변형을 처리하고 나면 예를 들어 다음과 같이 문장 표현이 달라지게 된다.

the boy's cars are different colors ---> the boy car be differ color

하지만 모든 단어는 명확한 의미의 차이가 있다. 단어의 어근을 찾는 것은 분석을 명확하게 하기 위해서이나 동시에 정확한 단어의 사용을 무시하게 되는 결과를 얻을 수도 있다. 단어의 기본형을 찾는 작업도 단어의 원래의 의미를 놓칠 가능성이 있다.

---

노트: 일반적으로 기본형을 찾는 작업은 단어의 어근을 찾는 것보다 먼저 수행된다

어근찾기와 기본형 찾기는 컴퓨터가 단어의 의미를 이해하는 것을 돕기 위해서 필요한 절차이다. 그러나 사람을 위해서는 원래의 표현을 그대로 살려두는 것이 필요할 수도 있다. 

파이선에서 제공하는 [자연처처리툴](http://www.nltk.org/)과 워드넷의 [기본형처리기](http://www.nltk.org/_modules/nltk/stem/wordnet.html)를 사용하여 단어를 변형하는 것을 설명하겠다.

In [4]:
import nltk
import collections

class WordNetLemmatizer(nltk.stem.WordNetLemmatizer):
		"""WordNetLemmatizer object. A wrapper around WordNet lemmatizer with a reverse lookup table."""
		def __init__(self, *args, **kwargs):
			super(self.__class__, self).__init__(*args, **kwargs)
			self._lem_memory = collections.defaultdict(set)
			# switch stem and memstem
			self._lem = self.lemmatize
			self.lem = self.memlem
	        
		def memlem(self, word):
			"""
			Wrapper around WordNetLemmatizer.lemmatize that remembers the original word.

			Args:
				word (string): word to lemmatize

			Returns:
				lemmatized word (string)
			"""
			lemmed_word = self._lem(word)
			self._lem_memory[lemmed_word].add(word)
			return lemmed_word
		    
		def unlem(self, lemmed_word):
			"""
			Unlemmatizes the lemmatized word.

			Args:
				lemmed_word (string): word to unstem  

			Returns:
				list of original words that were lemmatized to make lemmed word (list of strings). Remembering the exact original word requires storing more metadata.
			"""
			return sorted(self._lem_memory[lemmed_word], key=len)


파이선이 제공하는 [자연어처리툴](http://www.nltk.org/)과 함께 어근분석기인 [SnowCastle Stemmer](http://www.nltk.org/howto/stem.html)을 사용하겠다.

In [9]:
class SnowCastleStemmer(nltk.stem.SnowballStemmer):
		"""SnowCastleStemmer object. A wrapper around snowball stemmer with a reverse lookup table. Taken from https://gist.github.com/wassname/7fd4c975883074a99864"""
		def __init__(self, *args, **kwargs):
		    super(self.__class__, self).__init__(*args, **kwargs)
		    self._stem_memory = collections.defaultdict(set)
		    # switch stem and memstem
		    self._stem = self.stem
		    self.stem = self.memstem
		    
		def memstem(self, word):
			"""
			Wrapper around SnowCastleStemmer.stem that remembers the original word.

			Args:
				word (string): word to stem

			Returns:
				stemmed word (string)
			"""
			stemmed_word = self._stem(word)
			self._stem_memory[stemmed_word].add(word)
			return stemmed_word
		    
		def unstem(self, stemmed_word):
			"""
			Unstems the stemmed word.

			Args:
				stemmed_word (string): word to unstem  

			Returns:
				list of original words that were stemmed to make stemmed word (list of strings). Remembering the exact original word requires storing more metadata.
			"""
			return sorted(self._stem_memory[stemmed_word], key=len)

이제 기본형 찾기와 어근분석을 하여 단어를 어떻게 처리하는지를 살펴보겠다.

In [15]:
lemmatizer = WordNetLemmatizer()
stemmer = SnowCastleStemmer("english")

def lem_and_stem(word):
    lemmatized_word = lemmatizer.memlem(word)
    lemmatized_and_stemmed_word = stemmer.memstem(word)
    print("{} -> {} -> {}".format(word, lemmatized_word, lemmatized_and_stemmed_word))
    return lemmatized_and_stemmed_word

words = ["car", "cars", "car's", "cars'", "The", "quick", "brown", "fox", "jumped", "over", "the", "lazy", "dog"]
for word in words:
    lem_and_stem(word)

car -> car -> car
cars -> car -> car
car's -> car's -> car
cars' -> cars' -> car
The -> The -> the
quick -> quick -> quick
brown -> brown -> brown
fox -> fox -> fox
jumped -> jumped -> jump
over -> over -> over
the -> the -> the
lazy -> lazy -> lazi
dog -> dog -> dog


위의 경우는 단순한 단어들로서 크게 바뀌지 않는 경우이다. 좀 더 복잡한 문장의 예를 살펴보겠다.

In [19]:
sentences = ["the boy's cars are different colors", 
             "The goal of both stemming and lemmatization is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form"]
for sentence in sentences:
    words = sentence.split(" ")
    reduced_words = [lem_and_stem(word) for word in words]
    reduced_sentence = " ".join(reduced_words)
    print("\n{} -> {}\n".format(sentence, reduced_sentence))

the -> the -> the
boy's -> boy's -> boy
cars -> car -> car
are -> are -> are
different -> different -> differ
colors -> color -> color

the boy's cars are different colors -> the boy car are differ color

The -> The -> the
goal -> goal -> goal
of -> of -> of
both -> both -> both
stemming -> stemming -> stem
and -> and -> and
lemmatization -> lemmatization -> lemmat
is -> is -> is
to -> to -> to
reduce -> reduce -> reduc
inflectional -> inflectional -> inflect
forms -> form -> form
and -> and -> and
sometimes -> sometimes -> sometim
derivationally -> derivationally -> deriv
related -> related -> relat
forms -> form -> form
of -> of -> of
a -> a -> a
word -> word -> word
to -> to -> to
a -> a -> a
common -> common -> common
base -> base -> base
form -> form -> form

The goal of both stemming and lemmatization is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form -> the goal of both stem and lemmat is to reduc inflect form and sometim d

위의 결과를 보면 단어가 줄어든 것을 볼 수 있다.

위와 같은 과정을 거쳐서 축약된 문장으로부터 다시 원래의 문장을 복원하는 것은 어떻게 가능할까? 위의 예에서는 간단한 문장을 사용하였기 때문데 원래의 문장을 추정하는 것이 어느정도 쉽게 이루어질 수 있으나, 문장이 복잡한 경우는 원래의 문장을 제대로 복원하는 것이 어려운 작업이 된다.

In [35]:
def unlemstem(reduced_word, lemmatizer, stemmer):
        """
        Undoes the lemmatization and stemming process. 

        Args:
            token (string): lemmatized and stemmed token we want to convert to its original form(s)

        Returns:
            List of original form(s) of the token. (list of strings)
        """
        unstemmed = stemmer.unstem(reduced_word)
        print(unstemmed)
        originals = []
        for unstem in unstemmed:
            originals += lemmatizer.unlem(unstem)
            
        if len(originals) == 0 and len(unstemmed) != 0:
            # sometimes the lemmatizer does not remember words that were only stemmed
            originals = unstemmed
        
        print("{} <--- {}".format(reduced_word, originals))
        return originals
    
for sentence in sentences:
    words = sentence.split(" ")
    reduced_words = [lem_and_stem(word) for word in words]
    reduced_sentence = " ".join(reduced_words)
    print("\n{} -> {}\n".format(sentence, reduced_sentence))
            
    original_words = []
    for reduced_word in reduced_words:
        originals = unlemstem(reduced_word, lemmatizer, stemmer)
        sorted_originals = sorted(originals, key=len)
        longest_original = sorted_originals[-1] # if there are multiple words, choose the longest original word
        original_words.append(longest_original)
    original_sentence = " ".join(original_words)
    print("\n{} -> {}\n".format(reduced_sentence, original_sentence))

the -> the -> the
boy's -> boy's -> boy
cars -> car -> car
are -> are -> are
different -> different -> differ
colors -> color -> color

the boy's cars are different colors -> the boy car are differ color

['the', 'The']
the <--- ['the', 'The']
["boy's"]
boy <--- ["boy's"]
['car', 'cars', "car's", "cars'"]
car <--- ['car', 'cars', "car's", "cars'"]
['are']
are <--- ['are']
['different']
differ <--- ['different']
['colors']
color <--- ['colors']

the boy car are differ color -> The boy's cars' are different colors

The -> The -> the
goal -> goal -> goal
of -> of -> of
both -> both -> both
stemming -> stemming -> stem
and -> and -> and
lemmatization -> lemmatization -> lemmat
is -> is -> is
to -> to -> to
reduce -> reduce -> reduc
inflectional -> inflectional -> inflect
forms -> form -> form
and -> and -> and
sometimes -> sometimes -> sometim
derivationally -> derivationally -> deriv
related -> related -> relat
forms -> form -> form
of -> of -> of
a -> a -> a
word -> word -> word
to -> to

다른 방법으로, 어근찾기를 먼저 수행하고, 다음에 기본형 찾기를 수행할 수도 있다. 또한 여러가지 변형된 방법으로 텍스트를 분석해볼 수 있다. 텍스트 분석은 수치를 다루는 것이 아니므로, 다양한 문장에 대하여 기본적인 알고리즘을 적용하였을 때 실제로 어떻게 단어가 변형되는지를 확인함으로써 분석 경험을 쌓아야 한다.